this notebook sets up the data cleaning after submissions are pulled. Lines that are commented out were first tried here, and decided to be better placed in the intial function within the requests notebook

In [1]:
import pandas as pd

In [2]:
dem = pd.read_csv('./data/democrat140k.csv')
rep = pd.read_csv('./data/conservative140k.csv')

/home/dale/anaconda3/envs/python397/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (3,4,6,7,8,9,11,12,13,14,15,16,20,22,23,24,25,26,27,29,30,31,33,34,36,37,41,43,47,51,52,57,63,66,67,69,74,75,76,77,78,79,80,82,83,85,86,87,89,90,91,93,94,95,96,97,98) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/dale/anaconda3/envs/python397/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (11,22,84,85) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df = pd.concat([dem,rep])

In [4]:

df['removed_by_category'].value_counts()

automod_filtered    43152
moderator           27671
reddit              11019
deleted              2994
author                  5
Name: removed_by_category, dtype: int64

Removing posts that were filtered by automod (very likely bot posts), moderator (not following subreddit rules or potential bot), reddit (likely more serious), and author + deleted as the content of the post may have been poor or the author might not be active and would not be as valuable to predictions <br>
saving ID so that if comments are pulled later, comments for such posts can be filtered earlier on <br>

will likely be integrated into loop later on, so this doesnt have to be done for each pull, and since over half the posts are filtered, many more pulls will likely be required.

In [5]:
filter_cat = ['automod_filtered', 'moderator', 'reddit','author','deleted']


filtered_authors =  df[df['removed_by_category'].isin(filter_cat)]['author'].unique()

df =  df[~df['removed_by_category'].isin(filter_cat)]

In [6]:
filtered_authors = list(filtered_authors)

check to see if filtered authors still have active posts on reddit, so older submissions don't need to filtered by this list

In [7]:
df['author'].isin(filtered_authors).sum()


78317

Most of the filtered posts are from authors who still have active posts in subreddit<br>
Check to see if DF has no deleted posts

In [8]:
df['removed_by_category'].value_counts()

Series([], Name: removed_by_category, dtype: int64)

In [9]:
df = df[['selftext','created_utc','title','id','author','subreddit']]

In [10]:
df.isnull().sum()

selftext       120539
created_utc         0
title               0
id                  0
author              0
subreddit           0
dtype: int64

In [11]:
len(df)

132908

In [12]:
title_self = df[df['selftext'].notnull()]

In [13]:
title_self

,selftext,created_utc,title,id,author,subreddit
971,"Greetings, r/democrats. Recently we've receive...",1639631272,Announcement: Self/text posts are now disabled,rhj7bx,NickHancock,democrats
973,You’ve all heard them. But what’s gonna happen...,1639627277,How do anti-vax people expect to live normally...,rhhyin,DumplingsandDynamics,democrats
976,Democrats should be pushing the Build Back Bet...,1639615153,"Build Back Better, Kentucky!!!",rhe13t,notevergreens,democrats
987,He can forgive student loans. While he may ne...,1639597351,I am a parent scraping by who is about to take...,rh7hkg,hoyfkd,democrats
1009,[deleted],1639559096,Testing,rguxim,[deleted],democrats
...,...,...,...,...,...,...
94657,"Idk if this counts as a shit post, I hope it d...",1623900380,Just wanted to say I love the United States.,o1na5a,YPG6100,Conservative
94785,Here is your “Good News of the Week.” We are ...,1623880419,Good News of the Week!,o1glxw,saxman6257,Conservative
97633,Predictably have not gone down well in actual ...,1623349503,Biden’s Northern Ireland comments,nwunt2,ReluctantRev,Conservative
97783,Even third world Banana Republics laugh at thi...,1623331448,Guatemalan president says border crisis is Bid...,nwnkl8,Blueshound24,Conservative


Going to paste title and selftext since not all have selftext but could be useful context, remove [deleted] first

In [14]:
df = df[df['selftext'] != '[deleted]']

also check to see if title has any [deleted]


In [18]:
df = df[df['title'] != '[deleted]']

In [19]:
df['title_self'] = df['title'] + df['selftext']

In [20]:
df['sub_bin'] = df['subreddit'].map({'democrats':0, 'Conservative': 1})

In [21]:
df['sub_bin'].value_counts()

0    73533
1    56032
Name: sub_bin, dtype: int64

In [23]:
df.to_csv('./data/posts_cleaned_140k.csv')